In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [8]:
!pip install xlrd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.8 MB/s eta 0:00:00


In [6]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 9.6 MB/s eta 0:00:00


In [2]:
# Setting to use a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
# Setting to Use a High-RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 185.7 gigabytes of available RAM

You are using a high-RAM runtime!


In [9]:
#@title for Seoul to Seoul Trips
import pandas as pd
import os

# Path settings
folder_path = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
dongcode_path = '/content/drive/MyDrive/FMC_Data/dongcode_GISdongcode_dongname_ADMI_202303.xls'

# Read dongcode mapping file
dongcode_df = pd.read_excel(dongcode_path, dtype={'Dongcode': str})
dongcode_map = dongcode_df[['Dongcode', 'SGG_NM']].drop_duplicates()

def add_gu_columns(df):
    # Ensure codes are strings for matching
    df['O_ADMDONG_CD'] = df['O_ADMDONG_CD'].astype(str)
    df['D_ADMDONG_CD'] = df['D_ADMDONG_CD'].astype(str)
    # Merge to get O_gu
    df = df.merge(dongcode_map.rename(columns={'Dongcode':'O_ADMDONG_CD', 'SGG_NM':'O_gu'}),
                  on='O_ADMDONG_CD', how='left')
    # Merge to get D_gu
    df = df.merge(dongcode_map.rename(columns={'Dongcode':'D_ADMDONG_CD', 'SGG_NM':'D_gu'}),
                  on='D_ADMDONG_CD', how='left')
    return df

# File names
purposes = [f'purpose_{i}_with_OD_indicators' for i in range(1,8)]
files = [os.path.join(folder_path, f + '.csv') for f in purposes]

results = {}

for i, file in enumerate(files):
    # 1. Read file
    df = pd.read_csv(file, dtype={'O_ADMDONG_CD': str, 'D_ADMDONG_CD': str})
    # 2. Add gu columns
    df = add_gu_columns(df)
    # 3. Filter: only Seoul-to-Seoul
    df = df[(df['O_Seoul']==1) & (df['D_Seoul']==1)].copy()
    # 4. For calculation, ensure proper types
    df['MOVE_TIME'] = pd.to_numeric(df['MOVE_TIME'], errors='coerce')
    df['CNT'] = pd.to_numeric(df['CNT'], errors='coerce').fillna(0)

    total_trips = df['CNT'].sum()
    if total_trips == 0:
        # If no trips, fill with NaN or 0
        stats = [0]*7
    else:
        # % trips <= 15 min
        trips_15 = df[df['MOVE_TIME'] <= 15]['CNT'].sum() / total_trips * 100
        # % trips <= 30 min
        trips_30 = df[df['MOVE_TIME'] <= 30]['CNT'].sum() / total_trips * 100
        # % trips > 30 min
        trips_over_30 = df[df['MOVE_TIME'] > 30]['CNT'].sum() / total_trips * 100

        # % within same gu
        trips_same_gu = df[df['O_gu'] == df['D_gu']]['CNT'].sum() / total_trips * 100
        # % different gu
        trips_diff_gu = df[df['O_gu'] != df['D_gu']]['CNT'].sum() / total_trips * 100

        # % within same dong
        trips_same_dong = df[df['O_ADMDONG_CD'] == df['D_ADMDONG_CD']]['CNT'].sum() / total_trips * 100
        # % different dong
        trips_diff_dong = df[df['O_ADMDONG_CD'] != df['D_ADMDONG_CD']]['CNT'].sum() / total_trips * 100

        stats = [trips_15, trips_30, trips_over_30, trips_same_gu, trips_diff_gu, trips_same_dong, trips_diff_dong]

    results[f'purpose{i+1}'] = stats

# Create final DataFrame
index = [
    '% of trips <=15 min',
    '% of trips <=30 min',
    '% of trips > 30 min',
    '% of trips within the same gu',
    '% of trips coming from different gu',
    '% of trips within the same dong',
    '% of trips coming from different dong'
]
final_df = pd.DataFrame(results, index=index)
final_df.index.name = 'contents of percent of trips'

# Save to Excel
output_path = '/content/drive/MyDrive/FMC_Data/trip_percentages_by_purpose_intraSeoul.xlsx'
final_df.to_excel(output_path)
print(f"Saved output to {output_path}")

# Optionally display the table
final_df

Saved output to /content/drive/MyDrive/FMC_Data/trip_percentages_by_purpose.xlsx


,purpose1,purpose2,purpose3,purpose4,purpose5,purpose6,purpose7
contents of percent of trips,,,,,,,
% of trips <=15 min,27.085916,40.016979,38.433480,37.031979,58.572862,28.266251,43.450514
% of trips <=30 min,46.920184,58.266783,56.952696,60.151915,76.891095,48.980560,62.183371
% of trips > 30 min,53.079816,41.733217,43.047304,39.848085,23.108905,51.019440,37.816629
% of trips within the same gu,36.576574,59.525475,52.973159,50.029589,76.579710,36.731757,56.967769
% of trips coming from different gu,63.423426,40.474525,47.026841,49.970411,23.420290,63.268243,43.032231
% of trips within the same dong,11.667646,23.835718,19.446822,15.055541,9.309817,14.530919,21.290367
% of trips coming from different dong,88.332354,76.164282,80.553178,84.944459,90.690183,85.469081,78.709633


In [10]:
#@title for Everywhere to Seoul Trips

# import pandas as pd
# import os

# # Path settings
# folder_path = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
# dongcode_path = '/content/drive/MyDrive/FMC_Data/dongcode_GISdongcode_dongname_ADMI_202303.xls'

# # Read dongcode mapping file
# dongcode_df = pd.read_excel(dongcode_path, dtype={'Dongcode': str})
# dongcode_map = dongcode_df[['Dongcode', 'SGG_NM']].drop_duplicates()

# def add_gu_columns(df):
#     # Ensure codes are strings for matching
#     df['O_ADMDONG_CD'] = df['O_ADMDONG_CD'].astype(str)
#     df['D_ADMDONG_CD'] = df['D_ADMDONG_CD'].astype(str)
#     # Merge to get O_gu
#     df = df.merge(dongcode_map.rename(columns={'Dongcode':'O_ADMDONG_CD', 'SGG_NM':'O_gu'}),
#                   on='O_ADMDONG_CD', how='left')
#     # Merge to get D_gu
#     df = df.merge(dongcode_map.rename(columns={'Dongcode':'D_ADMDONG_CD', 'SGG_NM':'D_gu'}),
#                   on='D_ADMDONG_CD', how='left')
#     return df

# # File names
# purposes = [f'purpose_{i}_with_OD_indicators' for i in range(1,8)]
# files = [os.path.join(folder_path, f + '.csv') for f in purposes]

results = {}

for i, file in enumerate(files):
    # 1. Read file
    df = pd.read_csv(file, dtype={'O_ADMDONG_CD': str, 'D_ADMDONG_CD': str})
    # 2. Add gu columns
    df = add_gu_columns(df)
    # 3. Filter: Everywhere-to-Seoul
    df = df[df['D_Seoul']==1].copy()
    # 4. For calculation, ensure proper types
    df['MOVE_TIME'] = pd.to_numeric(df['MOVE_TIME'], errors='coerce')
    df['CNT'] = pd.to_numeric(df['CNT'], errors='coerce').fillna(0)

    total_trips = df['CNT'].sum()
    if total_trips == 0:
        # If no trips, fill with NaN or 0
        stats = [0]*7
    else:
        # % trips <= 15 min
        trips_15 = df[df['MOVE_TIME'] <= 15]['CNT'].sum() / total_trips * 100
        # % trips <= 30 min
        trips_30 = df[df['MOVE_TIME'] <= 30]['CNT'].sum() / total_trips * 100
        # % trips > 30 min
        trips_over_30 = df[df['MOVE_TIME'] > 30]['CNT'].sum() / total_trips * 100

        # % within same gu
        trips_same_gu = df[df['O_gu'] == df['D_gu']]['CNT'].sum() / total_trips * 100
        # % different gu
        trips_diff_gu = df[df['O_gu'] != df['D_gu']]['CNT'].sum() / total_trips * 100

        # % within same dong
        trips_same_dong = df[df['O_ADMDONG_CD'] == df['D_ADMDONG_CD']]['CNT'].sum() / total_trips * 100
        # % different dong
        trips_diff_dong = df[df['O_ADMDONG_CD'] != df['D_ADMDONG_CD']]['CNT'].sum() / total_trips * 100

        stats = [trips_15, trips_30, trips_over_30, trips_same_gu, trips_diff_gu, trips_same_dong, trips_diff_dong]

    results[f'purpose{i+1}'] = stats

# Create final DataFrame
index = [
    '% of trips <=15 min',
    '% of trips <=30 min',
    '% of trips > 30 min',
    '% of trips within the same gu',
    '% of trips coming from different gu',
    '% of trips within the same dong',
    '% of trips coming from different dong'
]
final_df = pd.DataFrame(results, index=index)
final_df.index.name = 'contents of percent of trips'

# Save to Excel
output_path = '/content/drive/MyDrive/FMC_Data/trip_percentages_by_purpose_All_to_Seoul.xlsx'
final_df.to_excel(output_path)
print(f"Saved output to {output_path}")

# Optionally display the table
final_df

Saved output to /content/drive/MyDrive/FMC_Data/trip_percentages_by_purpose_All_to_Seoul.xlsx


,purpose1,purpose2,purpose3,purpose4,purpose5,purpose6,purpose7
contents of percent of trips,,,,,,,
% of trips <=15 min,20.995292,34.495944,34.168300,29.737704,56.942164,17.191235,37.178560
% of trips <=30 min,37.310503,50.648948,51.653492,50.059668,74.886837,30.846933,54.111858
% of trips > 30 min,62.689497,49.351052,48.346508,49.940332,25.113163,69.153067,45.888142
% of trips within the same gu,27.272908,50.563246,45.989871,39.314815,74.373187,21.769941,47.704241
% of trips coming from different gu,72.727092,49.436754,54.010129,60.685185,25.626813,78.230059,52.295759
% of trips within the same dong,8.699848,20.246982,16.883207,11.831115,9.041570,8.612091,17.828341
% of trips coming from different dong,91.300152,79.753018,83.116793,88.168885,90.958430,91.387909,82.171659
